# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [40]:
# Load the libraries as required.
%load_ext dotenv
%dotenv
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [41]:
# features data frame
features_data = fires_dt.drop(columns= ["area"])
features_data


,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0


In [42]:
target_data = fires_dt["area"]


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [43]:
# creating numeric columns
features_data_numeric = features_data.drop(columns =["month", "day","coord_x", "coord_y"])
features_data_numeric

,ffmc,dmc,dc,isi,temp,rh,wind,rain
0,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0
1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0
2,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0
3,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2
4,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0
...,...,...,...,...,...,...,...,...
512,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0
513,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0
514,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0
515,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0


In [44]:
preproc1 = ColumnTransformer(
    transformers=[
    ("standardizer", StandardScaler(),features_data_numeric.columns),
    ("onehot", OneHotEncoder(handle_unknown='infrequent_if_exist'), ["month", "day"])
], remainder= "passthrough")
preproc1


ColumnTransformer(remainder='passthrough',
                  transformers=[('standardizer', StandardScaler(),
                                 Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                ('onehot',
                                 OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [46]:
preproc2 = ColumnTransformer(
    transformers=[
        ("standartizer",StandardScaler(),features_data_numeric),
        ("transformer", PowerTransformer(method='yeo-johnson')),
        ("one_hot", OneHotEncoder(),["month"])]
         ,remainder="passthrough")
preproc2

ColumnTransformer(remainder='passthrough',
                  transformers=[('standartizer', StandardScaler(),
                                      ffmc    dmc     dc   isi  temp  rh  wind  rain
0    86.2   26.2   94.3   5.1   8.2  51   6.7   0.0
1    90.6   35.4  669.1   6.7  18.0  33   0.9   0.0
2    90.6   43.7  686.9   6.7  14.6  33   1.3   0.0
3    91.7   33.3   77.5   9.0   8.3  97   4.0   0.2
4    89.3   51.3  102.2   9.6  11.4  99   1.8   0.0
..    ...    ...    ...   ...   ...  ..   ...   ...
512  81.6   56.7  665.6   1.9  27.8  32   2.7   0.0
513  81.6   56.7  665.6   1.9  21.9  71   5.8   0.0
514  81.6   56.7  665.6   1.9  21.2  70   6.7   0.0
515  94.4  146.0  614.7  11.3  25.6  42   4.0   0.0
516  79.5    3.0  106.7   1.1  11.8  31   4.5   0.0

[517 rows x 8 columns]),
                                ('transformer', PowerTransformer()),
                                ('one_hot', OneHotEncoder(), ['month'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [74]:
# Pipeline A = preproc1 + baseline
pipeline_a = Pipeline([
    ("preprocessing",preproc1),
    ("regressor", Ridge())
])
pipeline_a

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardizer',
                                                  StandardScaler(),
                                                  Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor', Ridge())])

In [62]:
#cross validation of pipeline_a
x = fires_dt.drop(columns=['area'])
y = fires_dt['area']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=123)


In [75]:
pipeline_a.fit(x_train, y_train)

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardizer',
                                                  StandardScaler(),
                                                  Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor', Ridge())])

In [76]:
Res_pipeline_a = cross_validate(
    pipeline_a,
    x_train,
    y_train,
    cv=5,
    scoring=['r2']
)
Res_pipeline_a = pd.DataFrame(Res_pipeline_a)
#Res_pipeline_a['test_neg_root_mean_squared_error'] = Res_pipeline_a["test_neg_root_mean_squared_error"].abs()
Res_pipeline_a

,fit_time,score_time,test_r2
0,0.009374,0.003445,-0.568754
1,0.004312,0.002688,-0.154584
2,0.003982,0.001939,-0.001814
3,0.003373,0.001567,-0.513243
4,0.003024,0.001478,-0.239529


In [65]:
Res_pipeline_a_aggr = Res_pipeline_a.agg(['mean','sem'])
Res_pipeline_a_aggr

,fit_time,score_time,test_r2
mean,0.005622,0.002607,-0.295585
sem,0.001705,0.000854,0.107546


In [36]:
# Pipeline B = preproc2 + baseline
pipeline_b = Pipeline([
    ("preprocessing", preproc2),
    ("regressor", LinearRegression())
])
pipeline_b

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standartizer',
                                                  StandardScaler(),
                                                       ffmc    dmc     dc   isi  temp  rh  wind  rain
0    86.2   26.2   94.3   5.1   8.2  51   6.7   0.0
1    90.6   35.4  669.1   6.7  18.0  33   0.9   0.0
2    90.6   43.7  686.9   6.7  14.6  33   1.3   0.0
3    91.7   33.3   77.5   9.0   8.3  97   4.0   0.2
4    89.3   51.3  102.2   9.6  11.4  99   1.8   0.0
..    ...    ...    ...   ...   ...  ..   ...   ...
512  81.6   56.7  665.6   1.9  27.8  32   2.7   0.0
513  81.6   56.7  665.6   1.9  21.9  71   5.8   0.0
514  81.6   56.7  665.6   1.9  21.2  70   6.7   0.0
515  94.4  146.0  614.7  11.3  25.6  42   4.0   0.0
516  79.5    3.0  106.7   1.1  11.8  31   4.5   0.0

[517 rows x 8 columns]),
                                                 ('transformer',
                                                  PowerTransformer()),
                                                 ('one_hot', OneHotEncoder(),
                                                  ['month'])])),
                ('regressor', LinearRegression())])

In [ ]:
# Pipeline C = preproc1 + advanced model


In [ ]:
# Pipeline D = preproc2 + advanced model

    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [66]:
pipeline_a.get_params()

{'memory': None,
 'steps': [('preprocessing',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('standardizer', StandardScaler(),
                                    Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                   ('onehot',
                                    OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                    ['month', 'day'])])),
  ('regressor', Ridge(alpha=1))],
 'verbose': False,
 'preprocessing': ColumnTransformer(remainder='passthrough',
                   transformers=[('standardizer', StandardScaler(),
                                  Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                 ('onehot',
                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                  ['month', 'day'])]),
 'regressor': Ridge(alpha=1),
 'preproc

In [ ]:

param_grid = {'regressor__alpha': [0.1, 1, 10, 100]}

grid_cv = GridSearchCV(
    estimator=pipeline_a,
    param_grid=param_grid,
    scoring='r2',
    cv=5,   
)
grid_cv

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('standardizer',
                                                                         StandardScaler(),
                                                                         Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor__alpha': [0.1, 1, 10, 100]}, scoring='r2')

In [90]:
grid_cv.get_params()

{'cv': 5,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('preprocessing',
   ColumnTransformer(remainder='passthrough',
                     transformers=[('standardizer', StandardScaler(),
                                    Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                   ('onehot',
                                    OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                    ['month', 'day'])])),
  ('regressor', Ridge())],
 'estimator__verbose': False,
 'estimator__preprocessing': ColumnTransformer(remainder='passthrough',
                   transformers=[('standardizer', StandardScaler(),
                                  Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                 ('onehot',
                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                          

In [91]:
grid_cv.fit(x_train, y_train)

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/compose/_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('standardizer',
                                                                         StandardScaler(),
                                                                         Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                                                        ('onehot',
                                                                         OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor__alpha': [0.1, 1, 10, 100]}, scoring='r2')

In [92]:
grid_cv.best_score_

-0.14327081756334964

In [95]:
best_model = grid_cv.best_estimator_
best_model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('standardizer',
                                                  StandardScaler(),
                                                  Index(['ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain'], dtype='object')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist'),
                                                  ['month', 'day'])])),
                ('regressor', Ridge(alpha=100))])

In [86]:
results = grid_cv.cv_results_
results = pd.DataFrame(results)
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005273,0.003943,0.002081,0.000848,0.1,{'regressor__alpha': 0.1},-0.559665,-0.172780,0.000150,-0.560385,-0.320186,-0.322573,0.218797,4
1,0.002551,0.000226,0.001316,0.000070,1.0,{'regressor__alpha': 1},-0.568754,-0.154584,-0.001814,-0.513243,-0.239529,-0.295585,0.215091,3
2,0.002233,0.000134,0.001201,0.000094,10.0,{'regressor__alpha': 10},-0.581718,-0.109500,-0.005183,-0.414384,-0.121522,-0.246461,0.215953,2
3,0.001956,0.000118,0.001021,0.000036,100.0,{'regressor__alpha': 100},-0.467187,-0.028105,-0.008619,-0.260596,0.048153,-0.143271,0.193248,1


By running gridsearch I have tried different aplha values to determine which one performs better. Results of thsi operations is that the parameter regressor alpha = 100 is scored as the best model

# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

In [98]:
import pickle
with open("ridge_model.pkl","wb") as file:
    pickle.dump(best_model,file)

print ("Best model saved")

Best model saved


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.